# Imports

In [8]:
import glob, pickle, datetime, numpy as np, math

# ML
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# from keras import backend as K
import scipy
from scipy import stats
import sys
from sklearn.preprocessing import QuantileTransformer
import sklearn

# Load Data

In [9]:
# Get List of Current Data Files
game = glob.glob("./Data/GameData*.pickle")
player = glob.glob("./Data/PlayerData*.pickle")
champ = glob.glob("./Data/ChampData*.pickle")

# Sort to find the most recent
game.sort()
player.sort()
champ.sort()

# Open Most Recent File
with open(game[-1], 'rb') as handle:
    gameData = pickle.load(handle)
    
with open(player[-1], 'rb') as handle:
    userData = pickle.load(handle)
    
with open(champ[-1], 'rb') as handle:
    champData = pickle.load(handle)

print("Loaded Data: " + datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y") + "\n{}\n{}\n{}".format(game[-1], player[-1], champ[-1]))

Loaded Data: 06:31PM on July 10, 2019
./Data\GameData.2019.07.10.pickle
./Data\PlayerData.2019.07.10.pickle
./Data\ChampData.2019.07.10.pickle


# Create Dataset

In [10]:
# Helper Function for removing Outliers
def removeOutliers(x, outlierConstant = 1.5):
    a = np.array(x)
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    for y in a.tolist():
        if y >= quartileSet[0] and y <= quartileSet[1]:
            resultList.append(y)
    return resultList

In [11]:
AllDataArray_X = []
AllDataArray_Y = []

# Find Mean and STD DEV of Mastery
count = 0

for player in userData:
    for champ in userData[player]['champions']:
        count += 1

# Find and graph z-scores of mastery
AllMastery = np.empty(count)
count = 0

for player in userData:
    for champ in userData[player]['champions']:
        AllMastery[count] = userData[player]['champions'][champ]['points']
        count += 1
        
MasteryMod = removeOutliers(AllMastery)
MasteryDev = np.std(MasteryMod)
MasteryMean = np.mean(MasteryMod)

# Create Array
for game in gameData:
    #Find Total Level and Rank
    totalLevel = 0
    totalRank = 1
    for user in game['players']:
        totalLevel += int(userData[user['accountID']]['level'])
        totalRank += int(userData[user['accountID']]['rank'])
    
    x = []
    for user in game['players']:
        #print(user)
        #print(user['championID'])
        try:
            x = x + [champData[user['championID']],
                np.cbrt((userData[user['accountID']]['champions'][user['championID']]['points'] - MasteryMean) / MasteryDev),
                userData[user['accountID']]['rank'] / totalRank,
                userData[user['accountID']]['level'] / totalLevel]
        except ValueError:
            print((userData[user['accountID']]['champions'][user['championID']]['points'] - MasteryMean) / MasteryDev)
        
                
    AllDataArray_X.append(x)
    AllDataArray_Y.append([int(game['winner']), int(not game['winner'])])

seperator = math.floor((len(AllDataArray_Y) - 1) / 4)

# Create Test and Training Data Sets
test_x = np.array(AllDataArray_X[0:seperator])
test_y = np.array(AllDataArray_Y[0:seperator])

train_x = np.array(AllDataArray_X[seperator:])
train_y = np.array(AllDataArray_Y[seperator:])

print("Done!")

Done!


# Create and Compile Model

In [23]:
model = keras.Sequential()

model.add(keras.layers.Dense(64, input_dim=40))
model.add(keras.layers.Activation('tanh'))
model.add(keras.layers.Dense(32))
model.add(keras.layers.Activation('tanh'))
model.add(keras.layers.Dense(2))
model.add(keras.layers.Activation('softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model Created")

Model Created


# Train Model

In [24]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.endLosses = [.5]
        self.endLossesX = [0]
        self.count = 0
        print("Started")

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('acc'))
        self.count += 1
        
    def on_epoch_end(self, batch, logs={}):
        self.endLosses.append(logs.get('val_acc'))
        self.endLossesX.append(self.count)

history = LossHistory()
myhistory = model.fit(train_x, train_y, 
                      epochs=20, verbose=1, 
                      validation_data=[test_x, test_y],
                      callbacks=[history, keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=1)])

Train on 76 samples, validate on 24 samples
Started
Epoch 1/20
76/76 [==============================] - 0s 4ms/sample - loss: 0.7776 - acc: 0.5132 - val_loss: 0.7254 - val_acc: 0.4583
Epoch 2/20
76/76 [==============================] - 0s 197us/sample - loss: 0.7254 - acc: 0.5263 - val_loss: 0.7126 - val_acc: 0.4583


# Evaluate Model

In [25]:
history = model.evaluate(test_x, test_y)

24/24 [==============================] - 0s 83us/sample - loss: 0.7126 - acc: 0.4583
